In [8]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [1]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [2]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [4]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [5]:
scenarios = ["Review with Question Method", "Review with Sound Method", " Review with Image Method",
             "Review with Meaning Method", "Learners choose the wrong answer", "Completing the Last Review Word",
             "Exit and Choose to Stay Scenario", "Scenario Learner decides to exit during review session before selecting an answer."]

In [6]:
prompt_uc = """
Review test
Main flow:
Step 1: Learners select the review button on the Home page. 
Step 2: The system redirects learners to the review's screen.
Step 3: System randomly selects one of four methods including questions, sound, image or meanings of one word for each words in learner's daily review words list with four selections of answer in the bottom of the screen. 
Step 4: Learners chooses by clicking on one of four selections.
Step 5: Learners click the right answer. 
Step 6: System shows a green word for learners to read. 
Step 7: This word is increased by the system in memory level.
Step 8: Learners press "Continue" button to go to the next word in the review words list.

Alternative flow 1: Learners choose the wrong answer
At step 5 of the basic flow: Learners click the wrong answer. 
Step 6: System shows a red word for learners to read. . 
Step 7: This word is decreased by the system in memory level.
Go back to step 8 in the basic flow and continue with the steps from step 8.

Alternative flow 2: Learners have done the review by coming to the last word in the review list
At step 8 of the basic flow: Learners press "Continue" button to go back to the Home page.
Step 9: System saved the review.

Alternative flow 3: Learners press "Exit" button and choose to stay
At step 4 of the basic flow: Learners press "Exit" button.
Step 5: System displays a pop up with two buttons "Confirm" and "Stay". 
Step 6: Learners choose "Stay".
Go back to step 4 in the basic flow and continue with the steps from step 4

Exception flow 1: Learners press "Exit" button and choose to exit
At step 3 of the basic flow: Learners press "Exit" button.
Step 4: System displays a pop up with two buttons "Confirm" and "Stay". 
Step 5: Learners choose "Confirm". 
Step 6: System redirect learners to Home page and the review will not be saved.

Pre condition:
User has login as learner. In their account, there are some words has been learned so there will be a review words list for today.

Post condition:
Update the status of vocabulary (memory level, review time) corresponding to the results of exercise checking. The status of the vocabulary to be reviewed is changed. The done review is saved and user could look back.
"""

In [8]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")

An error occurred with scenario 'Exit and Choose to Stay Scenario' iteration 1: 'list' object has no attribute 'items'


In [9]:
promptTestCase = [
{ "role": "system", "content": SYSTEM_PROMPT_8},
{ "role": "user", "content": scenarios[7] + "\n" + prompt_uc}
]
gpt_response = ask(promptTestCase, client, model)
json_data = json.loads(gpt_response)
write_json_to_txt_raw(json_data, scenarios[7] + "-Raw" + ".txt")

promptSelfRef = [
    { "role": "system", "content": SELF_REF},
    { "role": "user", "content": scenarios[7] + "\n" + gpt_response}
]
final_response = ask(promptSelfRef, client, model)
json_fin = json.loads(final_response)
write_json_to_txt(json_fin, scenarios[7] + ".txt")